In [3]:
import numpy as np
import pandas as pd
import os

notebook_path = os.path.abspath("")
os.chdir(notebook_path)

from nba_data_functions import *
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
pbp = pd.read_csv("/Users/yushunli/Documents/Data Science/NBA_Projects/Shot Ranking/data/pbpstats_2022.csv")
shot_detail = pd.read_csv("/Users/yushunli/Documents/Data Science/NBA_Projects/Shot Ranking/data/shotdetail_2022.csv")

In [8]:
def pbp_processing(pbp):
    """
    Process the initial play by play data to format the time columns and output a cleaner dataframe
    """
    #Convert ENDTIME into seconds
    pbp['END_TIME2'] = pd.to_datetime(pbp['ENDTIME'], format = '%M:%S')
    pbp['SECONDS_REMAINING'] = pbp['END_TIME2'].dt.second
    pbp['MINUTES_REMAINING'] = pbp['END_TIME2'].dt.minute
    pbp.drop(columns = ['END_TIME2', 'ENDTIME'], inplace = True)
    
    #Create new column for absolute score difference
    #pbp['ABS_SCORE_DIFF'] = abs(pbp['STARTSCOREDIFFERENTIAL'])

    #Filter for relevant rows then drop redundant rows with the same time stamps
    output = pbp[['GAMEID', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING', 'STARTSCOREDIFFERENTIAL', 'OPPONENT']]
    output =  output.groupby(['GAMEID', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING', 'OPPONENT'], as_index = False).max()
    #Rename the STARTSCOREDIFFERENTIAL column to START_SCORE_DIFF
    output.rename(columns = {'STARTSCOREDIFFERENTIAL': 'START_SCORE_DIFF'}, inplace = True)
    return output.sort_values(by = ['GAMEID', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING'], ascending = [True, True, False, False])

In [9]:
def process_score_difference(game_df):
    """ 
    Output a new dataframe that contains the score difference at each second of the game.
    2. Create a new df that contains the score difference at each second of the game based on the "TIME_REMAINING" column
    """
    if game_df.PERIOD.max() > 4:
        max_game_seconds = 48*60 + (game_df.PERIOD.max() - 4)*5*60
    else: 
        max_game_seconds = 48*60
    
    #Create new empty dataframee with a row for each second of game time
    score_diff = pd.DataFrame()
    score_diff['TIME_ELAPSED'] = range(0, max_game_seconds+1)
    score_diff['GAME_ID'] = game_df.GAMEID.unique()[0]

    #Create column for Time Elapsed in game_df as follows:
    # If PERIOD is betwee 1 - 4: time elapsed = (PERIOD-1)*12*60 + (12*60) - SECONDS_REMAINING - MINUTES_REMAINING*60
    # If PERIOD is greater than 4: time elapsed = 48*60 + (PERIOD-5)*5*60 + (5*60) - SECONDS_REMAINING - MINUTES_REMAINING*60
    game_df['TIME_ELAPSED'] = np.where(game_df['PERIOD'] <= 4, (game_df['PERIOD']-1)*12*60 + (12*60) - game_df['SECONDS_REMAINING'] - game_df['MINUTES_REMAINING']*60, 48*60 + (game_df['PERIOD']-5)*5*60 + (5*60) - game_df['SECONDS_REMAINING'] - game_df['MINUTES_REMAINING']*60)
    #Create two new columns for the 2 teams in the game: 'Team1' and 'Team2'
    game_df['Team1'] = game_df['OPPONENT'].unique()[0]
    game_df['Team2'] = game_df['OPPONENT'].unique()[1]

    #Join the two dataframes on the TIME_ELAPSED column
    score_diff = score_diff.merge(game_df, how = 'left', on = 'TIME_ELAPSED')
    #Set the ABS_SCORE_DIFF at TIME_ELAPSED = 0 to be 0
    score_diff.loc[score_diff['TIME_ELAPSED'] == 0, 'START_SCORE_DIFF'] = 0
    #Fill rows with a missing ABS_SCORE_DIFF with the last known value backwards and then forwards
    score_diff['START_SCORE_DIFF'] = score_diff['START_SCORE_DIFF'].fillna(method = 'bfill')
    score_diff['START_SCORE_DIFF'] = score_diff['START_SCORE_DIFF'].fillna(method = 'ffill')

    return score_diff[['GAME_ID', 'TIME_ELAPSED', 'START_SCORE_DIFF', 'OPPONENT', 'Team1', 'Team2']]

In [10]:
def pbp_game_processing(pbp):
    """
    This function will parse through every game in the pbp dataset (based on unique GAMEID) and then run the subsequent pbp_processing function on each game.
    """
    pbp_list = []
    #Parse through each game using a for loop
    for game in pbp.GAMEID.unique():
        pbp_game = pbp.loc[pbp['GAMEID'] == game]
        #Run the pbp_processing function on each game
        df1 = pbp_processing(pbp_game)
        pbp_scores = process_score_difference(df1)
        pbp_list.append(pbp_scores)
    pbp_processed = pd.concat(pbp_list)
    return pbp_processed

In [17]:
shot_detail_df = shot_detail_time_elapsed(shot_detail)
#pbp_df = pbp_processing(pbp)
#pbp_df['STARTSCOREDIFFERENTIAL'] = pbp_df['STARTSCOREDIFFERENTIAL'].fillna(method = 'ffill')
shot_detail_df.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM', 'TIME_ELAPSED'],
      dtype='object')

In [23]:
shot_detail_df.tail()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,TIME_ELAPSED
215601,Shot Chart Detail,22200956,624,203078,Bradley Beal,1610612764,Washington Wizards,5,1,56,...,Less Than 8 ft.,3,-36,-1,1,0,20230304,WAS,TOR,3064
215602,Shot Chart Detail,22200956,628,1628398,Kyle Kuzma,1610612764,Washington Wizards,5,1,23,...,24+ ft.,27,6,277,1,0,20230304,WAS,TOR,3097
215603,Shot Chart Detail,22200956,643,1626153,Delon Wright,1610612764,Washington Wizards,5,0,32,...,Less Than 8 ft.,3,-36,13,1,0,20230304,WAS,TOR,3148
215604,Shot Chart Detail,22200956,647,204001,Kristaps Porzingis,1610612764,Washington Wizards,5,0,29,...,Less Than 8 ft.,6,2,63,1,0,20230304,WAS,TOR,3151
215605,Shot Chart Detail,22200956,660,203078,Bradley Beal,1610612764,Washington Wizards,5,0,10,...,24+ ft.,29,190,232,1,0,20230304,WAS,TOR,3170


In [24]:
test_game = shot_detail_df.loc[shot_detail_df.GAME_ID == 22200956].sort_values(by = ['TIME_ELAPSED'], ascending = [True])
test_game.head(10)

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,TIME_ELAPSED
215531,Shot Chart Detail,22200956,8,204001,Kristaps Porzingis,1610612764,Washington Wizards,1,11,36,...,Less Than 8 ft.,2,-2,23,1,1,20230304,WAS,TOR,24
201058,Shot Chart Detail,22200956,10,1628384,O.G. Anunoby,1610612761,Toronto Raptors,1,11,19,...,24+ ft.,23,-238,9,1,0,20230304,WAS,TOR,41
201059,Shot Chart Detail,22200956,12,1630567,Scottie Barnes,1610612761,Toronto Raptors,1,11,15,...,Less Than 8 ft.,5,-56,4,1,0,20230304,WAS,TOR,45
215532,Shot Chart Detail,22200956,14,204001,Kristaps Porzingis,1610612764,Washington Wizards,1,11,2,...,24+ ft.,29,-161,242,1,1,20230304,WAS,TOR,58
201060,Shot Chart Detail,22200956,16,1627783,Pascal Siakam,1610612761,Toronto Raptors,1,10,43,...,Less Than 8 ft.,6,23,66,1,0,20230304,WAS,TOR,77
215533,Shot Chart Detail,22200956,20,204001,Kristaps Porzingis,1610612764,Washington Wizards,1,10,29,...,8-16 ft.,15,-142,54,1,1,20230304,WAS,TOR,91
201061,Shot Chart Detail,22200956,23,1630567,Scottie Barnes,1610612761,Toronto Raptors,1,10,7,...,8-16 ft.,9,-65,66,1,0,20230304,WAS,TOR,113
215534,Shot Chart Detail,22200956,26,204001,Kristaps Porzingis,1610612764,Washington Wizards,1,9,59,...,Less Than 8 ft.,4,31,36,1,0,20230304,WAS,TOR,121
201062,Shot Chart Detail,22200956,29,1627832,Fred VanVleet,1610612761,Toronto Raptors,1,9,53,...,Less Than 8 ft.,4,36,29,1,0,20230304,WAS,TOR,127
201063,Shot Chart Detail,22200956,31,1627783,Pascal Siakam,1610612761,Toronto Raptors,1,9,48,...,Less Than 8 ft.,0,0,0,1,1,20230304,WAS,TOR,132


In [60]:
def shotdetail_game_processing(game_shot_detail):
    ''' 
    This function will process the play by play data for each game and will output a clean dataframe that tracks each possession and the score difference before and after the possession
    '''
    pbp_df = game_shot_detail.copy()
    #Reset the index of the dataframe
    pbp_df.reset_index(inplace = True)

    #Drop columns that are not needed: LOC_X, LOC_Y, EVENT_TYPE, 'ACTION_TYPE', 'SHOT_TYPE','SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE'
    pbp_df.drop(columns = ['index','GRID_TYPE','LOC_X', 'LOC_Y', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE','SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'GAME_DATE'], inplace = True)

    '''Possession Counter'''
    #Create a new column called 'POSSESSION' and the first row (by first GAME_EVENT_ID) will be 1
    pbp_df.loc[pbp_df.GAME_EVENT_ID == pbp_df.GAME_EVENT_ID.min(), 'POSSESSION'] = 1
    #Run a for loop through the pbp_df starting at the second row
    for i in range(1, len(pbp_df)):
        #If the i'th row has the same TEAM_NAME as the (i-1)'th row, then the i'th row is part of the same possession as the (i-1)'th row
        #If the i'th row has a different TEAM_NAME as the (i-1)'th row, then the i'th row is part of a new possession
        if pbp_df.loc[i, 'TEAM_NAME'] == pbp_df.loc[i-1, 'TEAM_NAME']:
            pbp_df.loc[i, 'POSSESSION'] = pbp_df.loc[i-1, 'POSSESSION']
        else:
            pbp_df.loc[i, 'POSSESSION'] = pbp_df.loc[i-1, 'POSSESSION'] + 1
    
    return pbp_df

shotdetail_game_test = shotdetail_game_processing(test_game)

In [41]:
test_pbp = pbp.loc[pbp.GAMEID == 22200956]
test_pbp.head()

,ENDTIME,EVENTS,FG2A,FG2M,FG3A,FG3M,GAMEDATE,GAMEID,NONSHOOTINGFOULSTHATRESULTEDINFTS,OFFENSIVEREBOUNDS,OPPONENT,PERIOD,SHOOTINGFOULSDRAWN,STARTSCOREDIFFERENTIAL,STARTTIME,STARTTYPE,TURNOVERS,DESCRIPTION,URL
467752,00:06,MISS Beal 30' 3PT Pullup Jump Shot\nTrent Jr. ...,0,0,1,0,2023-03-04,22200956,0,0,TOR,5,0,-7,00:14,Off FT Make,0,MISS Beal 30' 3PT Pullup Jump Shot,https://videos.nba.com/nba/pbp/media/2023/03/0...
467753,00:06,MISS Beal 30' 3PT Pullup Jump Shot\nTrent Jr. ...,0,0,1,0,2023-03-04,22200956,0,0,TOR,5,0,-7,00:14,Off FT Make,0,Trent Jr. REBOUND (Off:0 Def:5),https://videos.nba.com/nba/pbp/media/2023/03/0...
467754,00:24,MISS Wright 4' Layup: Anunoby BLOCK (1 BLK)\nW...,2,0,0,0,2023-03-04,22200956,0,1,TOR,5,0,-4,00:43,Off Corner 3 Make,0,MISS Wright 4' Layup: Anunoby BLOCK (1 BLK),https://videos.nba.com/nba/pbp/media/2023/03/0...
467755,00:24,MISS Wright 4' Layup: Anunoby BLOCK (1 BLK)\nW...,2,0,0,0,2023-03-04,22200956,0,1,TOR,5,0,-4,00:43,Off Corner 3 Make,0,WIZARDS Rebound,NaN
467756,00:24,MISS Wright 4' Layup: Anunoby BLOCK (1 BLK)\nW...,2,0,0,0,2023-03-04,22200956,0,1,TOR,5,0,-4,00:43,Off Corner 3 Make,0,MISS Porzingis 6' Alley Oop Layup,https://videos.nba.com/nba/pbp/media/2023/03/0...


In [45]:
def pbp_game_processing(game_pbp):
    ''' 
    This function will process the play by play data for each game and will output a clean dataframe that tracks each possession and the score difference before and after the possession
    '''
    pbp_df = game_pbp.copy()
    #Reset the index of the dataframe
    pbp_df.reset_index(inplace = True)

    #Drop columns that are not needed: URL, GAMEDATE
    pbp_df.drop(columns = ['URL', 'GAMEDATE', 'index', 'EVENTS'], inplace = True)

    return pbp_df

pbp_game_processing(test_pbp)

,ENDTIME,FG2A,FG2M,FG3A,FG3M,GAMEID,NONSHOOTINGFOULSTHATRESULTEDINFTS,OFFENSIVEREBOUNDS,OPPONENT,PERIOD,SHOOTINGFOULSDRAWN,STARTSCOREDIFFERENTIAL,STARTTIME,STARTTYPE,TURNOVERS,DESCRIPTION
0,00:06,0,0,1,0,22200956,0,0,TOR,5,0,-7,00:14,Off FT Make,0,MISS Beal 30' 3PT Pullup Jump Shot
1,00:06,0,0,1,0,22200956,0,0,TOR,5,0,-7,00:14,Off FT Make,0,Trent Jr. REBOUND (Off:0 Def:5)
2,00:24,2,0,0,0,22200956,0,1,TOR,5,0,-4,00:43,Off Corner 3 Make,0,MISS Wright 4' Layup: Anunoby BLOCK (1 BLK)
3,00:24,2,0,0,0,22200956,0,1,TOR,5,0,-4,00:43,Off Corner 3 Make,0,WIZARDS Rebound
4,00:24,2,0,0,0,22200956,0,1,TOR,5,0,-4,00:43,Off Corner 3 Make,0,MISS Porzingis 6' Alley Oop Layup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,10:42,1,0,0,0,22200956,0,0,WAS,1,0,-5,11:02,Off Arc 3 Make,0,Kuzma REBOUND (Off:0 Def:1)
362,11:11,1,0,1,0,22200956,0,1,WAS,1,0,-2,11:36,Off At Rim Make,0,MISS Anunoby 24' 3PT Jump Shot
363,11:11,1,0,1,0,22200956,0,1,WAS,1,0,-2,11:36,Off At Rim Make,0,Barnes REBOUND (Off:1 Def:0)
364,11:11,1,0,1,0,22200956,0,1,WAS,1,0,-2,11:36,Off At Rim Make,0,MISS Barnes 6' Putback Layup


In [46]:
stats_df = pd.read_csv('data/nbastats_2022.csv')
stats_df.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,22200001,2,12,0,1,7:36 PM,12:00,NaN,Start of 1st Period (7:36 PM EST),NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0
1,22200001,4,10,0,1,7:36 PM,12:00,Jump Ball Horford vs. Embiid: Tip to Harris,NaN,NaN,...,76ers,PHI,5,202699,Tobias Harris,1.610613e+09,Philadelphia,76ers,PHI,1
2,22200001,7,2,86,1,7:36 PM,11:38,Brown BLOCK (1 BLK),NaN,MISS Embiid 13' Turnaround Fadeaway Shot,...,NaN,NaN,4,1627759,Jaylen Brown,1.610613e+09,Boston,Celtics,BOS,1
3,22200001,9,4,0,1,7:36 PM,11:35,NaN,NaN,Tucker REBOUND (Off:1 Def:0),...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,1
4,22200001,10,5,40,1,7:36 PM,11:35,NaN,NaN,Tucker Out of Bounds Lost Ball Turnover (P1.T1),...,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,1


In [56]:
test_stats = stats_df.loc[stats_df.GAME_ID == 22200956]
#test_stats.sort_values(by = ['EVENTNUM'], ascending = [True])

In [55]:
def pbp_game_processing2(game_pbp):
    ''' 
    This function will process the play by play data for each game and will output a clean dataframe that tracks each possession and the score difference before and after the possession
    '''
    pbp_df = game_pbp.copy()
    #Reset the index of the dataframe
    pbp_df.reset_index(inplace = True)

    #Drop columns that are not needed: VIDEO_AVAILABLE_FLAG, 'WCTIMESTRING', 'PCTIMESTRING', 'PERSON3TYPE','PLAYER3_ID', 'PLAYER3_NAME', 'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_CITY','PLAYER3_TEAM_NICKNAME', 'PLAYER3_TEAM_ABBREVIATION',
    pbp_df.drop(columns = ['VIDEO_AVAILABLE_FLAG', 'WCTIMESTRING', 'PCTIMESTRING', 'PLAYER1_ID', 'PLAYER1_TEAM_ID', 'PLAYER1_TEAM_CITY','PLAYER1_TEAM_NICKNAME',
                           'PLAYER2_ID', 'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_CITY','PLAYER2_TEAM_NICKNAME',
                           'PERSON3TYPE','PLAYER3_ID', 'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_CITY','PLAYER3_TEAM_NICKNAME','index'], inplace = True)

    return pbp_df

test_stats_game = pbp_game_processing2(test_stats)

In [61]:
shotdetail_game_test.head()

,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,HTM,VTM,TIME_ELAPSED,POSSESSION
0,22200956,8,204001,Kristaps Porzingis,1610612764,Washington Wizards,1,11,36,1,1,WAS,TOR,24,1.0
1,22200956,10,1628384,O.G. Anunoby,1610612761,Toronto Raptors,1,11,19,1,0,WAS,TOR,41,2.0
2,22200956,12,1630567,Scottie Barnes,1610612761,Toronto Raptors,1,11,15,1,0,WAS,TOR,45,2.0
3,22200956,14,204001,Kristaps Porzingis,1610612764,Washington Wizards,1,11,2,1,1,WAS,TOR,58,3.0
4,22200956,16,1627783,Pascal Siakam,1610612761,Toronto Raptors,1,10,43,1,0,WAS,TOR,77,4.0


In [62]:
test_stats_game.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION
0,22200956,2,12,0,1,NaN,Start of 1st Period (5:11 PM EST),NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN
1,22200956,4,7,4,1,NaN,NaN,Poeltl Violation:Jump Ball (R.Mott),NaN,NaN,5.0,Jakob Poeltl,TOR,0,NaN,NaN,NaN,NaN
2,22200956,5,10,0,1,Jump Ball Porzingis vs. Poeltl: Tip to Kuzma,NaN,NaN,NaN,NaN,4.0,Kristaps Porzingis,WAS,5,Jakob Poeltl,TOR,Kyle Kuzma,WAS
3,22200956,8,1,75,1,Porzingis 2' Driving Finger Roll Layup (2 PTS)...,NaN,NaN,0 - 2,2,4.0,Kristaps Porzingis,WAS,4,Delon Wright,WAS,NaN,NaN
4,22200956,10,2,1,1,NaN,NaN,MISS Anunoby 24' 3PT Jump Shot,NaN,NaN,5.0,O.G. Anunoby,TOR,0,NaN,NaN,NaN,NaN


In [64]:
#Read txt file as a dataframe
pd.read_csv('/Users/yushunli/Documents/Data Science/nba-rating-calculator/Play_by_Play.txt', sep = '\t')


,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Option2,Option3,Team_id,Person1,Person2,Person3,Team_id_type,Person1_type,Person2_type,Person3_type
0,006728e4c10e957011e1f24878e6054a,2,12,1,546410,7200,0,0,0,0,1473d70e5646a26de3c52aa1abd85b1f,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,0,0,0,0
1,006728e4c10e957011e1f24878e6054a,4,10,1,546420,7200,0,0,0,0,01be0ad4af7aeb1f6d2cc2b6b6d6d811,8d2127290c94bd41b82a2938734bc750,99104de2626f67c1fa2ce70504970c3f,766802a8fda500d7945950de7398c9c6,2,4,5,4
2,006728e4c10e957011e1f24878e6054a,7,1,1,546590,7020,79,2,0,0,01be0ad4af7aeb1f6d2cc2b6b6d6d811,42e0d7167f04a4ff958c6442da0e6851,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,4,0,0
3,006728e4c10e957011e1f24878e6054a,8,6,1,546620,7020,2,0,0,1,01be0ad4af7aeb1f6d2cc2b6b6d6d811,616281dee946056b071699476fdee9ec,42e0d7167f04a4ff958c6442da0e6851,0b1c631effab25d4af2d16fc123c3d31,2,5,4,1
4,006728e4c10e957011e1f24878e6054a,10,3,1,546810,7020,10,1,0,0,01be0ad4af7aeb1f6d2cc2b6b6d6d811,42e0d7167f04a4ff958c6442da0e6851,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37884,ff42c1cba89fd3f3a7a6711a11d5576d,682,9,4,604090,152,1,0,0,0,45ba8fc87f55b1191c50c400dc7ed11c,c9ef395e3fd4c1dda4a4427fd86ec1d6,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,3,3,0,0
37885,ff42c1cba89fd3f3a7a6711a11d5576d,683,8,4,604620,152,0,0,0,0,45ba8fc87f55b1191c50c400dc7ed11c,5cce6ffa455e6372d9de0de400482ab6,44230324724c84f122ac62a5f0918314,0370a0d090da0d0edc6319f120187e0e,3,5,5,0
37886,ff42c1cba89fd3f3a7a6711a11d5576d,685,5,4,604960,119,2,1,0,0,45ba8fc87f55b1191c50c400dc7ed11c,48ec4e6c52f418d5ca4ef510ba473ea0,618f6d58ab2881152607c2a6e057bc51,0370a0d090da0d0edc6319f120187e0e,3,5,4,0
37887,ff42c1cba89fd3f3a7a6711a11d5576d,687,13,4,605110,0,0,0,0,0,01be0ad4af7aeb1f6d2cc2b6b6d6d811,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,0370a0d090da0d0edc6319f120187e0e,2,0,0,0
